In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm_notebook

from torch.utils.data import DataLoader, Dataset, Sampler
from torch.utils.data.dataloader import default_collate

# Визуализация инициализаций

In [ ]:
def xavier_uniform_boundaries(n_in, n_out):
    bound = (np.sqrt(6)/(np.sqrt(n_in) + np.sqrt(n_out)))
    return (-bound, bound)

In [ ]:
def xavier_normal_boundaries(n_in, n_out):
    bound = (np.sqrt(2)/(np.sqrt(n_in) + np.sqrt(n_out)))
    return (-bound, bound)

In [ ]:
bounds_list = []
for i in range(1, 100):
    bounds = xavier_uniform_boundaries(i, i)
    bounds_list.append(bounds)

In [ ]:
plt.bar(list(range(1, len(bounds_list)+1)), [2*x[1] for x in bounds_list], bottom=[x[0] for x in bounds_list])

In [ ]:
bounds_list = []
for i in range(1, 100):
    bounds = xavier_normal_boundaries(i, i)
    bounds_list.append(bounds)

In [ ]:
plt.bar(list(range(1, len(bounds_list)+1)), [2*x[1] for x in bounds_list], bottom=[x[0] for x in bounds_list])

# DataLoader in PyTorch

__DataLoader__ -- класс в PyTorch, который позволяет итеративно проходить по датасету, отвечает за оркестрацию всего процесса работы с датасетом.

In [ ]:
DataLoader(
  dataset, 
  batch_size=1, 
  shuffle=False, 
  sampler=None, 
  batch_sampler=None, 
  num_workers=0, 
  collate_fn=None, 
  pin_memory=False, 
  drop_last=False, 
  timeout=0, 
  worker_init_fn=None, 
  prefetch_factor=2, 
  persistent_workers=False
  )

- __dataset__ -- позволяет создать кастомные классы для работы с датасетом, где можно указать логику формирвоания батча.
- __sampler__ -- определяет порядок элементов из датасета, которые будут идти в батч, то есть это список индексов, объединенных в батч. Удобно переопределять, когда обучение распредленное.  
- __collate_fn__ -- позволяет сделать финальную предобработку над батчем данных. Если, например, в батч попали последовательности разных размеров, то после уже сбора батча, можно будет дополнить последовательности нулями относительно максимально длиной последовательности.



## Custom Dataset

In [ ]:
import torch

In [ ]:
BATCH_SIZE = 128
EPOCHS = 100

In [ ]:
class CustomDataset(Dataset):
    # Конструктор, где считаем датасет
    def __init__(self, dataset_path):
        with open(dataset_path, 'rb') as f:
            self.X, self.target = pickle.load(f)

        return
    
    # Переопределяем метод вычисление размера датасета
    def __len__(self):
        return len(self.X)

    # Переопределяем метод,
    # который достает по индексу наблюдение из датасет
    def __getitem__(self, idx):
        return self.X[idx], self.target[idx]

## Custom Sampler

In [ ]:
class CustomSampler(Sampler):

    # Конструктор, где инициализируем индексы элементов
    def __init__(self, data):
        self.data_indices = np.arange(len(data))

        shuffled_indices = np.random.permutation(len(self.data_indices))

        self.data_indices = np.ascontiguousarray(self.data_indices)[shuffled_indices]

        return

    def __len__(self):
        return len(self.data_indices)

    # Возращает итератор,
    # который будет возвращать индексы из перемешанного датасета
    def __iter__(self):
        return iter(self.data_indices)

## Custom collate_fn

In [ ]:
def collate(batch):
    ans = default_collate(batch)
    return ans

In [ ]:
def create_data_loader(train_dataset, train_sampler,
                       test_dataset, test_sampler):
    train_loader = DataLoader(dataset=train_dataset, sampler=train_sampler,
                              batch_size=BATCH_SIZE, collate_fn=collate,
                              shuffle=False)

    test_loader = DataLoader(dataset=test_dataset, sampler=test_sampler,
                             batch_size=BATCH_SIZE, collate_fn=collate,
                             shuffle=False)

    return train_loader, test_loader

In [ ]:
!git clone https://github.com/RiskModellingResearch/DeepLearning_Autumn22.git

In [ ]:
# Создаем объекты Custom Dataset и Sampler
train_ds = CustomDataset('DeepLearning_Autumn22/week_03/data/X_train_cat.pickle')
train_sampler = CustomSampler(train_ds.X)

test_ds = CustomDataset('DeepLearning_Autumn22/week_03/data/X_test_cat.pickle')
test_sampler = CustomSampler(test_ds.X)

In [ ]:
train_loader, test_loader = create_data_loader(train_ds, train_sampler, 
                                               test_ds, test_sampler)

In [ ]:
def run_train():
    for epoch in tqdm_notebook(range(EPOCHS)):
        for features, labels in train_loader:
            pass
    return

In [ ]:
run_train()